# Część trzecia
<div class="alert alert-block alert-danger">
<b>Uwaga:</b> Należy upewnić się że w prawym górnym rogu tego okna wybrany jest kernel "edugen" a nie "Python 3"
</div>

***
## Format VCF
***

Przyjętym standardowym formatem plików przechowującym informację o wariantach jest format <a href="http://samtools.github.io/hts-specs/VCFv4.3.pdf">VCF (Variant Call Format)</a>. W takich plikach przechowywana jest informacja o wszystkich wariantach odnalezionych przez program do variant callingu. Pliki VCF możemy podzielić na trzy sekcje:
- metadanych (@meta)
- informacje o wariatach (@fix)
- informacje o genotypie (@gt)

<img src="png/vcf.png" width="400">


In [ ]:
! head -n 50 sandbox/motherHC.vcf

***
### Region metadanych (@meta)
Znajduje się na początku pliku i zawiera informacje (poprzedzone znakami "##") dotyczące programu który wygenerował plik VCF oraz parametrów używanych przez ten program, czy też na temat skrótów stoowanych w innych miejscach pliku. Dane w tej sekcji są zapisywane w kolejnych wierszach. Pozostałe regiony mają formę tabeli.

### Region informacji o wariancie (@fix)
Region ten obejmuje pierwszych 8 kolumn, które zawierają informację o konkretnym wariancie. Dane te są wspólne dla wszystkich wariantów i obejmują:
- pozycję chromosomową (kolumny "#CHROM" i "POS")
- identyfikator wariantu, np. z bazy dbSNP (kolumna "ID")
- allel referencyjny i zmieniony (kolejno kolumny "REF" i "ALT") - uwaga, liczba zmienionych alleli może być większa niż 1, sa wtedy zapisywane po przecinku
- informację o jakości danego wariantu (kolumna "QUAL")
- informację czy wariant przeszedł filtr jakości (kolumna "FILTER")

### Region informacji o genotypie (@gt)
W tym regionie znajdziemy informację związane z danym wariantem dla poszczególnych próbek. W 10 kolumnie poszczególne wartości sa oddzielone od siebie dwukropkiem, a informację co która wartość oznacza znajdują się w kolumnie 9 ("FORMAT"). Najczęsciej są to informacje o genotypie, ilości odczytów w danym miejscu czy ich jakości, a wyjaśnienie stosowanych skrótów i co oznaczają znajdziemy w regionie metadanych. Jeśli w pliku VCF opisanych jest kilka próbek, to informacje te dla kolejnych próbek są zapisywane w kolejnych kolumnach. Pola które zwyczajowo znajdują się w tym regionie to:
- GT (Genotype) - genotyp w danym miejscu, oddzielony znakami "/" lub "|". Wartość 0 oznacza allel referencyjny, 1 oznacza pierwszy zmieniony allel z pola "ALT", 2 drugi allel itd. Kombinacja 0/1 oznacza heterozygotyczna zmianę, 1/1 zmianę homozygotyczną
- AD (Allel Read Depth) - liczba odczytów zawierających dany wariant
- DP (Read Depth) - ogólna liczba odczytów w danym miejscu

| #CHROM | POS    | ID    | REF | ALT | QUAL | FILTER | INFO | FORMAT   | SAMPLE_1_ID           | SAMPLE_2_ID               |
|--------|--------|-------|-----|-----|------|--------|------|----------|-----------------------|---------------------------|
| 22     | 160036 | snp_1 | A   | C   | .    | PASS   | .    | GT:AD:DP | 0/1:7:30              | 1/1:31:31                 |
| 22     | 160408 | snp_2 | T   | C   | .    | PASS   | .    | GT:AD:DP | ./.:.:.               | 0/1:10:21                 |
| 22     | 160612 | snp_2 | C   | G   | .    | PASS   | .    | GT:AD:DP | 0/1:22:53             | 1/1:41:41                 |

***
## Dekompozycja i normalizacja plików VCF
***
Pomimo tego, że VCF jest standardowym formatem przechowywania danych o wariantach to warianty mogą być przechowywane w różnej postaci. W celu unifikacji tych informacji należy skorzystac z <a href="https://genome.sph.umich.edu/wiki/Vt">narzędzia Vt</a>. Pozwala ono na rozbicie multi-allelicznych wariantów na poszczególne wiersze. Dodatkowo ujednolica sposób zapisu wariantów typu INDEL, tak aby były przesunięte możliwie najbardziej na "lewo" oraz możliwie najbardziej "oszczędne" w zapisie.
<img src="png/Normalizacja.png">

***
Najpierw nalezy przeprowadzić dekompozycję, czyli rozbicie wariantów multi-allelicznych a następnie normalizację, czyli ujednolicenie zapisu wariantów.

In [ ]:
! vt decompose -s sandbox/motherHC.vcf | vt normalize -r ref/ref.fasta - > sandbox/motherHC.vt.vcf

***
## Eksploracja plików VCF
***
Narzędziem służącym do manipulowania plikami VCF jest `bcftools`. Program ten posiada <a href="https://samtools.github.io/bcftools/bcftools.html">wiele opcji</a>, możemy na przykład filtrować warianty, wydzielać regiony czy podsumować dane. Na początek porównajmy pliki przed i po obróbce programem Vt.

<div class="alert alert-block alert-warning">
    <b>Zadanie:</b> Policz ile wariantów znajduje się w plikach <b>motherHC.vcf</b> i <b>motherHC.vt.vcf</b>. Czy w pliku źródłowym były warianty multialleliczne? Użyj poleceń bash - <i>cat</i>, <i>grep</i> i <i>wc</i> 
</div>

In [ ]:
! cat sandbox/motherHC.vcf | grep -v "^#" | wc -l
! cat sandbox/motherHC.vt.vcf | grep -v "^#" | wc -l

Możemy również podejrzeć ile jest wariantów typu snp lub indel. Służy do tego komenda `view` z argumentem `-v`.

In [ ]:
! bcftools view -v snps sandbox/motherHC.vcf | grep -v "^#" | wc -l
! bcftools view -v indels sandbox/motherHC.vcf | grep -v "^#" | wc -l

### Poszukiwanie wariantów wspólnych lub unikalnych
Przydatną funkcją może być poszukiwanie wariantów które występują w dwóch lub większej liczbie próbek, lub wariantów unikalnych dla danej próbki. Służy do tego komenda `isec` z pakietu `bcftools` o której więcej możemy <a href="https://samtools.github.io/bcftools/bcftools.html#isec">poczytać tutaj</a>. Domyślnie program daje wyniki w postaci wariantów unikalnych dla konkretnych próbek oraz warianty wspólne dla wszystkich.

In [ ]:
! bcftools isec -p sandbox/isec vcf/sampleA.vcf.gz vcf/sampleB.vcf.gz 

Wyniki znajdziemy w folderze `sandbox/isec`, w którym znajdziemy 4 pliki, których opis znajduje się w pliku 